In [ ]:
import json
import sklearn
import numpy as np
import pandas as pd

from tqdm import tqdm

from itertools import product
from collections.abc import Iterable

from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit
)
from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    roc_auc_score,
    f1_score
)
from sklearn.cross_decomposition import PLSRegression

from imblearn.over_sampling import SMOTE

In [ ]:
### **import json**
  json은 JSON 데이터를 읽고 쓸 수 있게 해주는 표준 라이브러리입니다.

### **import sklearn**
  sklearn(scikit learn)은 파이썬 머신러닝 라이브러리로 다양한 머신러닝 알고리즘을 제공합니다.

### **import numpy as np**
  배열을 처리하는 라이브러리로 다차원 배열의 빠른 처리와 배열 broadcasting 기능, 선형 변환 기능을 사용할 수 있습니다. numpy 라이브러리 기능을 사용할 때, 편의성을 위해 np로 호출할 수 있도록 as np를 붙여줍니다.

### **import pandas as pd**
  Pandas는 Series와 DataFrame 자료형을 사용하여 데이터를 처리하는 파이썬 데이터 분석 라이브러리입니다. pandas 함수 호출을 간편하게 처리하기 위해 as pd를 사용합니다.

### **from tqdm import tqdm**
  tqdm은 파이썬의 반복문의 진행 상황을 시각적으로 보여주는 라이브러리입니다.

### **from itertools import product**  
  itertools.product를 사용하여 반복 가능한 변수들의 곱을 계산할 수 있도록 해줍니다. 즉, 반복 가능한 변수들 간의 모든 조합을 생상하는 데 사용할 수 있습니다.

### **from collections.abc import Iterable**
  Iterable은 객체가 반복 가능한지를 확인할 수 있도록 해주는 클래스입니다.

### **from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit**
  StratifiedKFold는 데이터를 계층화된 방법으로 분할하여 교차 검증을 수행할 수 있도록 해주는 클래스입니다.

  StratifiedShuffleSplit는 불균형 분포를 위한 교차 검증을 위해서 계층화된 방식으로 무작위 분할을 통해 Train/Test Set을 생성하는 클래스입니다.

### **from sklearn.metrics import precision_score, recall_score,   accuracy_score, roc_auc_score, f1_score**

  **Precision, Recall, Accurcay, ROC_AUC** 점수모델의 성능을 평가하기 위한 다양한 메트릭을 계산하는 함수들을 import합니다.

### **from sklearn.cross_decomposition import PLSRegression**

  부분 최소 제곱 회귀함수를 수행하는 클래스입니다. 고차원 데이터를 사용하여 선형 회귀를 진행하기 위한 클래스입니다.

### **from imblearn.over_sampling import SMOTE**

  소수 클래스 집합의 데이터를 오버샘플링하기 위한 SMOTE 기법을 사용하는 클래스입니다. 불균형 데이터셋을 처리하기 위해서 소수 클래스의 샘플을 합성하여 추가하는 SMOTE 방식을 사용할 수 있습니다.



In [ ]:
def data_split(X, y, seed):
    sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = seed)

    for train_idx, test_idx in sss.split(X, y):
        train_x = X.iloc[train_idx].reset_index(drop = True)
        train_y = y.iloc[train_idx].reset_index(drop = True)
        test_x = X.iloc[test_idx].reset_index(drop = True)
        test_y = y.iloc[test_idx].reset_index(drop = True)

    return train_x, test_x, train_y, test_y

**data_split** 함수는 데이터를 훈련 세트와 테스트 세트로 분할하기 위한 함수입니다. 이 함수는 **데이터셋 X(특성들), y(타겟 변수),** 그리고 **시드(seed)** 값을 입력으로 받습니다.

**StratifiedShuffleSplit**: 이 클래스는 데이터를 계층화하여 무작위로 섞은 후 훈련 세트와 테스트 세트로 분할합니다.

 아래는 StratifiedShuffleSplit 함수의 입력 파라미터에 대한 설명입니다.
 ```python
 sklearn.model_selection.StratifiedShuffleSplit(n_splits=10, *, test_size=None, train_size=None, random_state=None)

  n_splitsint, default=10
   - 반복과 재구성, 분할 횟수

  test_size floatorint, default= none
   - 테스트 분할에 포함할 데이터 집합의 비율을 나타냅니다.
     Integer 타입의 경우, 세트스 샘플의 절대 수를 나타내고, Float 타입의 경우는 테스트 분할 비율을 의미합니다.

  random_stateint, RandomState instance or None, default=None
   - 훈련과 테스트 셋의 무작위성을 제어합니다. seed를 고정하여 무작위 분할이 아닌 재현 가능한 결과를 만들어 낼 수 있도록 해줍니다.

 # 더 자세한 내용은 (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)을 참조하세요.
 ```

## 함수 동작 과정
1. **StratifiedShuffleSplit 객체 생성**: 데이터를 한 번 분할하고, 테스트 데이터 비율을 20%로 설정합니다.
2. **split 호출**: X는 데이터의 특성, y는 라벨입니다. split 함수는 X와 y를 분할한 후 train_idx와 test_idx를 반환합니다.
3. **인덱스를 이용한 데이터 분할**: train_idx와 test_idx를 이용해 train_x, test_x, train_y, test_y를 할당합니다. `reset_index(drop=True)`로 인덱스를 재설정합니다.
4. **데이터 반환**: 분할된 train_x, test_x, train_y, test_y를 반환합니다.


**parameter_grid** 함수는 Python Dictionary 형색의 입력 매개변수(**param_dict**)를 받아서 가능한 모든 매개변수 조합을 생성합니다. 위 함수는 머신러닝 모델의 Hyperparameter 최적화 과정에서 사용할 수 있습니다.

```python
  isinstance(object, type)
  
  ininstance() 함수는 입력받은 object가 입력받은 type 유형이라면 True, 그렇지 않다면 False를 반환하는 함수입니다.
```

 * 위 isinstance함수를 사용하여 사전 형식인지를 검사하고. 만약 사전 타입이 아니라면 TypeError를 발생시킵니다.

 * 만약 입력받은 param_dict가 사전 형식이라면, 사전의 각 값이 반복 가능한 객체(Iterable)인지 검사합니다. 반복 불가능한 값이 있다면 역시 TypeError를 발생시킵니다.

 이후, 입력받은 param_dict 사전의 키와 값들을 정렬하여 튜플로 분리합니다.

아래는 product()함수에 대한 설명입니다.
 ```python
  itertools.product(*iterables, repeat=1)

  입력 받은 반복 가능한 변수들의 데카르트 곱을 도출하는 함수입니다,

  위 함수는 중첩된 for-루프와 동등합니다. 예를 들어, product(A, B)는 ((x,y) for x in A for y in B)와 같은 것을 반환합니다.
  # 더 자세한 내용은 (https://docs.python.org/ko/3/library/itertools.html)을 참조하세요
 ```

 ## 함수 동작 과정
1. 들어온 입력이 dictionary가 아니면 TypeError를 발생시킵니다.
2. dictionary의 value가 iterable이 아니면 TypeError를 발생시킵니다.
    - iterable이란 list, tuple, set, dict, str 와 같이 맴버들을 한 번에 한 개씩 반환할 수 있는 객체를 말합니다.
    - int, float, bool 등의 객체는 iterable 하지 않습니다.

3. dictionary의 항목들을 key를 기준으로 정렬합니다.
4. dictionary의 key와 value를 각각 keys와 values에 저장합니다.
5. product(*values)를 사용하여 각 키에 대한 가능한 모든 값의 조합을 생성하고, 이 조합들을 사전 형식으로 다시 매핑하여 결과 리스트(params_grid)에 추가합니다.
6. params_grid를 반환합니다.


In [ ]:
def multiclass_cross_validation(model, x, y, seed):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

    metrics = ['precision', 'recall', 'f1', 'accuracy']

    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))

    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []

    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []

    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]

        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            onehot_train_y = pd.get_dummies(train_y)

            model.fit(train_x, onehot_train_y)

            train_pred = np.argmax(model.predict(train_x), axis = 1)
            val_pred = np.argmax(model.predict(val_x), axis = 1)

        else:
            model.fit(train_x, train_y)

            train_pred = model.predict(train_x)
            val_pred = model.predict(val_x)

        train_precision.append(precision_score(train_y, train_pred, average = 'macro'))
        train_recall.append(recall_score(train_y, train_pred, average = 'macro'))
        train_f1.append(f1_score(train_y, train_pred, average = 'macro'))
        train_accuracy.append(accuracy_score(train_y, train_pred))

        val_precision.append(precision_score(val_y, val_pred, average = 'macro'))
        val_recall.append(recall_score(val_y, val_pred, average = 'macro'))
        val_f1.append(f1_score(val_y, val_pred, average = 'macro'))
        val_accuracy.append(accuracy_score(val_y, val_pred))

    result = dict(zip(train_metrics + val_metrics,
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy),
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy)]))

    return(result)

**multiclass_cross_validation** 함수는 다중 클래스 분류 문제에 대한 교차 검증을 수행하는 함수입니다.이 함수는 **모델(model), 특성 데이터(x), 타겟 데이터(y),** 그리고 **시드 값(seed)**을 입력으로 받습니다.

아래는 **StratifiedKFold**에 대한 설명입니다.

```python
 sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)

 위 StratifiedKFold는 Train/Test 분할을 위한 indices를 제공합니다.
 이 Cross Validation은 KFold 방식의 변형으로 각 클래스에 대한 비율을 보존하여 KFold를 진행하게 됩니다.

 아래는 입력 Parameter에 대한 설명입니다.

  n_splitsint, default=5
  - Folds의 숫자를 정하는 파라미터로 최소한 2개 이상의 정수를 가져야 한다.

  shufflebool, default=False
  - split 이전에 각 클래스의 샘플들을 섞을지 결정하는 인자입니다. 다만, 분할 내의 샘플들을 섞이지 않습니다.

  random_stateint, RandomState instance or None, default=None
   - shuffle이 참아라면, random_state의 값에 따라서 랜덤 인덱스의 순서에 영향을 미치며, Random_state의 int 값에 따라서 값이 고정되어 재현이 가능해집니다. shuffle을 사용하지 않는다면, None 값을 사용합니다.

 # 더 자세한 내용은 (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)를 참고하세요.
```

**multiclass_cross_validation** 은 모델과 데이터를 받아서 5-fold cross validation을 수행하고, 각 fold에 대한 평균 precision, recall, f1, accuracy를 계산하여 반환하는 함수입니다. 이때, k-fold cross validation을 수행하기 위해서 StratifiedKFold를 사용합니다.

## 함수 동작 과정
1. **Fold 평가 변수 초기화**: 각 fold에 대한 평가를 저장하기 위한 변수(리스트)를 선언합니다.
2. **데이터 분할**: `skf.split(x, y)`는 x와 y를 5-fold로 나누고, 각 fold의 train 및 validation index를 반환합니다.
3. **데이터 추출 및 모델 학습**:
   - PLSRegression은 다중 클래스 분류를 지원하지 않기에 one-hot encoding을 필요로 합니다. 다른 모델은 일반적인 방식으로 학습합니다.
   - 예측값을 처리할 때, `argmax`를 사용하여 가장 높은 확률을 가진 클래스를 선택합니다.
4. **모델 예측 및 평가 지표 계산**:
   - train_x, val_x에 대한 예측을 수행합니다.
   - 각 metric은 `average='macro'`로 설정하여 클래스별 평균을 계산합니다.

## 반환 형태
- 반환되는 result는 각 평가 지표의 평균값을 담은 dictionary입니다.
```python
{
    'train_precision': 0.95,
    'train_recall': 0.94,
    'train_f1': 0.94,
    'train_accuracy': 0.95,
    'val_precision': 0.92,
    'val_recall': 0.91,
    'val_f1': 0.91,
    'val_accuracy': 0.92
}
```

In [ ]:
def binary_cross_validation(model, x, y, seed):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

    metrics = ['precision', 'recall', 'f1', 'accuracy', 'auc']

    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))

    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []
    train_auc = []

    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []
    val_auc = []

    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]

        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            model.fit(train_x, train_y)

            train_pred_score = model.predict(train_x)
            train_pred = np.where(train_pred_score < 0.5, 0, 1).reshape(-1)

            val_pred_score = model.predict(val_x)
            val_pred = np.where(val_pred_score < 0.5, 0, 1).reshape(-1)

        else:
            model.fit(train_x, train_y)

            train_pred = model.predict(train_x)
            train_pred_score = model.predict_proba(train_x)[:, 1]

            val_pred = model.predict(val_x)
            val_pred_score = model.predict_proba(val_x)[:, 1]

        train_precision.append(precision_score(train_y, train_pred))
        train_recall.append(recall_score(train_y, train_pred))
        train_f1.append(f1_score(train_y, train_pred))
        train_accuracy.append(accuracy_score(train_y, train_pred))
        train_auc.append(roc_auc_score(train_y, train_pred_score))

        val_precision.append(precision_score(val_y, val_pred))
        val_recall.append(recall_score(val_y, val_pred))
        val_f1.append(f1_score(val_y, val_pred))
        val_accuracy.append(accuracy_score(val_y, val_pred))
        val_auc.append(roc_auc_score(val_y, val_pred_score))

    result = dict(zip(train_metrics + val_metrics, 줘
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy), np.mean(train_auc),
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy), np.mean(val_auc)]))

    return(result)

binary_cross_validation은 모델과 데이터를 받아서 5-fold cross validation을 수행하고, 각 fold에 대한 평균 precision, recall, f1, accuracy, auc를 계산하여 반환하는 함수입니다.

이진 분류를 수행한다는 점을 제외하면, 기본적으로 multiclass_cross_validation과 동일한 과정으로 동작합니다.
- 이진 분류를 수행하기 때문에, 들어오는 데이터의 레이블도 기본적으로 이진 형태를 가집니다.

k-fold cross validation을 수행하기 위해서 StratifiedKFold를 사용합니다.
- StratifiedKFold는 각 fold에 대해서 클래스의 비율이 유지되도록 데이터를 분할합니다.

평가를 수행하는 metric은 아래와 같습니다.
1. precision
2. recall
3. f1
4. accuracy
5. auc
    - auc는 roc_auc_score를 사용하여 계산합니다.
    - 다중 클래스 분류와 달리, 이진 분류는 roc_auc_score를 사용하여 auc를 계산할 수 있습니다.


각 fold에 대한 평가를 저장하기 위한 변수(리스트)를 선언합니다.


skf.split(x, y)는 x와 y를 5-fold로 분할한 후, 각 fold에 대한 train index와 validation index를 반환하는 함수입니다.
해당되는 index를 사용하여 train_x, train_y, val_x, val_y를 추출하고, 모델을 학습시킵니다.


모델을 학습시킨 후, train_x, val_x에 대한 예측값을 계산합니다.
- 모델이 PLSRegression인 경우, predict 함수를 사용하여 예측값을 계산하고, 예측값이 0.5보다 작으면 0, 0.5보다 크면 1로 변환합니다.
- 모델이 PLSRegression이 아닌 경우, predict 함수를 사용하여 예측값을 계산하고, predict_proba 함수를 통해 양성 클래스에 대한 예측 확률을 반환합니다.

각 성능 metric 리스트에 대해 np.mean을 사용하여 평균을 계산합니다.


반환될 result는 다음과 형태를 가집니다.
```python
{
    'train_precision': 0.95,
    'train_recall': 0.94,
    'train_f1': 0.94,
    'train_accuracy': 0.95,
    'train_auc': 0.99,
    'val_precision': 0.92,
    'val_recall': 0.91,
    'val_f1': 0.91,
    'val_accuracy': 0.92,
    'val_auc': 0.98
}
```

In [ ]:
def binary_smote_cross_validation(model, x, y, seed, args):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

    metrics = ['precision', 'recall', 'f1', 'accuracy', 'auc']

    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))

    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []
    train_auc = []

    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []
    val_auc = []

    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]

        smote = SMOTE(random_state = args.smoteseed, k_neighbors = args.neighbor)
        train_x, train_y = smote.fit_resample(train_x, train_y)

        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            model.fit(train_x, train_y)

            train_pred_score = model.predict(train_x)
            train_pred = np.where(train_pred_score < 0.5, 0, 1).reshape(-1)

            val_pred_score = model.predict(val_x)
            val_pred = np.where(val_pred_score < 0.5, 0, 1).reshape(-1)

        else:
            model.fit(train_x, train_y)

            train_pred = model.predict(train_x)
            train_pred_score = model.predict_proba(train_x)[:, 1]

            val_pred = model.predict(val_x)
            val_pred_score = model.predict_proba(val_x)[:, 1]

        train_precision.append(precision_score(train_y, train_pred))
        train_recall.append(recall_score(train_y, train_pred))
        train_f1.append(f1_score(train_y, train_pred))
        train_accuracy.append(accuracy_score(train_y, train_pred))
        train_auc.append(roc_auc_score(train_y, train_pred_score))

        val_precision.append(precision_score(val_y, val_pred))
        val_recall.append(recall_score(val_y, val_pred))
        val_f1.append(f1_score(val_y, val_pred))
        val_accuracy.append(accuracy_score(val_y, val_pred))
        val_auc.append(roc_auc_score(val_y, val_pred_score))

    result = dict(zip(train_metrics + val_metrics,
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy), np.mean(train_auc),
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy), np.mean(val_auc)]))

    return(result)

**binary_smote_cross_validation** 함수는 이진 분류 문제에 대해 SMOTE 기법을 활용하여 교차 검증을 수행하는 함수입니다. **모델(model), 특성 데이터(x), 타겟 데이터(y), 시드 값(seed), 추가 인자(args)**를 입력으로 받습니다.

 앞서 multiclass_cross_valdation 함수에서 사용했던 **StratifiedKFold**를 사용하여 데이터를 5개의 폴드로 나눕니다. 이를 통해 각 클래스에 대한 비율을 유지한 체로 폴드를 생성됩니다.

 이후 각 폴드를 구성하는 데이터 셋에 대해 SMOTE 기법을 사용하여 훈련 데이터의 소수 클래스를 **오버샘플링**합니다. 이를 통해 불균형 데이터섯에 대한 처리를 진행할 수 있습니다.

이후, binary_cross_validation, multiclass_cross_validation에서 사용한 방식과 동일하게, 각 폴드에서 모델을 훈련시키고, 훈련 및 검증 데이터에 대한 여러 평가 지표(정밀도, 재현율, F1 점수, 정확도, AUC)를 계산합니다. 또한, 각 폴드의 평가 지표들의 평균을 계산하여 최종 결과를 사전 형태로 반환합니다.

In [ ]:
# 사용되지 않는 함수입니다.
# def CV(x, y, model, params, seed):
#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

#     metrics = ['precision', 'recall', 'f1', 'accuracy']

#     train_metrics = list(map(lambda x: 'train_' + x, metrics))
#     val_metrics = list(map(lambda x: 'val_' + x, metrics))

#     train_precision = []
#     train_recall = []
#     train_f1 = []
#     train_accuracy = []

#     val_precision = []
#     val_recall = []
#     val_f1 = []
#     val_accuracy = []

#     for train_idx, val_idx in skf.split(x, y):
#         train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
#         val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]

#         try:
#             clf = model(random_state = seed, **params)
#         except:
#             clf = model(**params)


#         if model == sklearn.cross_decomposition._pls.PLSRegression:
#             onehot_train_y = pd.get_dummies(train_y)

#             clf.fit(train_x, onehot_train_y)

#             train_pred = np.argmax(clf.predict(train_x), axis = 1)
#             val_pred = np.argmax(clf.predict(val_x), axis = 1)

#         else:
#             clf.fit(train_x, train_y)

#             train_pred = clf.predict(train_x)
#             val_pred = clf.predict(val_x)

#         train_precision.append(precision_score(train_y, train_pred, average = 'macro'))
#         train_recall.append(recall_score(train_y, train_pred, average = 'macro'))
#         train_f1.append(f1_score(train_y, train_pred, average = 'macro'))
#         train_accuracy.append(accuracy_score(train_y, train_pred))

#         val_precision.append(precision_score(val_y, val_pred, average = 'macro'))
#         val_recall.append(recall_score(val_y, val_pred, average = 'macro'))
#         val_f1.append(f1_score(val_y, val_pred, average = 'macro'))
#         val_accuracy.append(accuracy_score(val_y, val_pred))

#     result = dict(zip(['params'] + train_metrics + val_metrics,
#                       [params] + [np.mean(train_precision),
#                                   np.mean(train_recall),
#                                   np.mean(train_f1),
#                                   np.mean(train_accuracy),
#                                   np.mean(val_precision),
#                                   np.mean(val_recall),
#                                   np.mean(val_f1),
#                                   np.mean(val_accuracy)]))

#     return(result)

In [ ]:
def metric_mean(data, metric: str):
    mean_per_hp = list(map(lambda x: np.mean(x[1]), data[metric].items()))
    return mean_per_hp

**metric_mean** 함수는 주어진 데이터에서 특정 메트릭의 평균값을 계산하는 함수입니다.

data 매개변수는 평가 메트릭이 저장된 데이터 구조(예: 사전)를 받습니다.

metric 매개변수를 통해 계산하고자 하는 특정 메트릭의 이름을 문자열로 받아 data[metric].items()를 통해 해당 메트릭에 대한 모든 값들을 순회합니다.

각 값에 대해 numpy의 mean 함수를 사용하여 평균을 계산하고, 이를 리스트로 변환합니다.

결과적으로, 입력된 메트릭에 대한 평균값들의 리스트를 반환합니다.

In [ ]:
def print_best_param(val_result, metric: str):

    mean_list = metric_mean(val_result, metric)
    max_idx = mean_list.index(max(mean_list))

    best_param = val_result['model'][f'model{max_idx}']

    return best_param

**print_best_param**은 성능이 가장 좋은 하이퍼파라미터를 결정하는 데에 사용합니다.

모델의 검증 결과를 담고 있는 데이터를 val_result 파라미터로 받고, metric는 최적화하고자 하는 평가 지표의 이름을 문자열로 받습니다.

함수는 먼저 **metric_mean**함수를 호출하여 주어진 메트릭에 대한 각 매개변수 설정의 평균 점수를 계산합니다.

이후, 가장 높은 평균 점수를 가진 매개변수 설정의 인덱스(max_idx)를 찾고 val_result에서
해당 인덱스를 사용하여 최적의 매개변수를 검색하고 반환합니다.

In [ ]:
def load_val_result(path: str, tg_num: int, inhale_type: str, model: str, is_smote = True):
    if is_smote:
        try:
            with open(f'{path}/tg{tg_num}_val_results/binary_smote5/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
        except:
            with open(f'{path}tg{tg_num}_val_results/binary_smote5/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
    else:
        try:
            with open(f'{path}/tg{tg_num}_val_results/binary/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
        except:
            with open(f'{path}tg{tg_num}_val_results/binary/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)

    return val_result

**load_val_result** 함수는 JSON 형식의 검증 결과를 불러오는 함수입니다.

path, tg_num, inhale_type, model 매개변수들은 파일 경로와 파일 이름을 결정하는 데 사용되며 is_smote 매개변수는 SMOTE 기법을 사용한 결과를 불러올지, 사용하지 않은 결과를 불러올 지 결정해줍니다.

SMOTE 사용 여부(is_smote)에 따라 다른 파일 경로에서 JSON 파일을 불러옵니다.
try-except 블록을 통해 첫 번째 try에서 파일을 열지 못하면, except 블록에서 다른 경로의 파일을 시도합니다.

JSON 파일의 내용은 json.load 함수를 사용하여 파이썬 객체로 변환됩니다.

In [ ]:
# 사용되지 않는 함수입니다.
# def print_best_param(path: str, tg_num: int, inhale_type: str, model: str, metric: str):
#     data = load_val_result(path, tg_num, inhale_type, model)

#     mean_list = metric_mean(data, 'f1')
#     max_idx = mean_list.index(max(mean_list))

#     best_param = data['model'][f'model{max_idx}']

#     return best_param